<a href="https://colab.research.google.com/github/lucasfelipecdm/fiap-hackathon-vision-guard/blob/main/fiap_hackathon_visionguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FIAP VisionGuard

##Grupo 24 - 02/2025

Esse Jupyter file, contém informações e passo a passo de como treinar um modelo YOLOv8 com custom dataset para identificação de facas, tesouras e outros objetos cortantes.

## Instalação de bibliotecas necessárias.

Foram utilizadas a biblioteca de ultralytics para importar o modelo YOLO, email validator para validar o formato do e-mail de segurança, opencv para processamento do video, desenho das bounding boxes e salvar os frames com objetos cortantes e yagmail para envio dos e-mails.

In [2]:
!pip install ultralytics
!pip install opencv-python
!pip install email_validator
!pip install yagmail

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Importação das bibliotecas

Aqui importamos as bibliotecas que utilizaremos durante a detecção dos objetos e envio do e-mail, também conectamos ao Google Drive para utilização dos datasets e salvar os frames.

In [3]:
import os
import cv2
import yagmail

from ultralytics import YOLO
from email_validator import validate_email, EmailNotValidError
from google.colab import drive
from google.colab import userdata

drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


## Criação e configuração de caminhos

Aqui criamos algumas variáveis para facilitar a chamada dos caminhos durante o resto do código.

In [4]:
data_path = userdata.get('DATA_PATH')
dataset_path = data_path + '/dataset3'
video_path = data_path + '/test-video/video-test-1.mp4'
video_path2 = data_path + '/test-video/video-test-2.mp4'
output_folder = dataset_path + '/results' # Cada dataset tem sua própria pasta de resultados
os.makedirs(output_folder, exist_ok=True)  # Cria pasta caso não exista


## Criação e configuração do email

Configura os dados de envio de e-mail e também o e-mail do receptor (ou da central de segurança)

In [5]:
SENDER_EMAIL = userdata.get('GMAIL_EMAIL')
SENDER_PASSWORD = userdata.get('GMAIL_PASSWORD')
RECEIVER_EMAIL = "lucasfelipecdm@hotmail.com" # E-mail da central de segurança vem aqui

## Configuração das classes de objetos a serem detectadas

Aqui configuramos a array com os valores que gostaríamos que fossem identificas e notificadas a central de segurança, como utilizamos o dataset com somente label de faca vamos deixar somente esse item para ser notificado, mas poderíamos adicionar outros objetos como: tesoura, estilete, cutelos, facões, etc.

In [6]:
CLASSES_TO_DETECT = ['knife']

## Definição da função de treinamento

Aqui criamos a função que realizará o treinamento do modelo utilizando o dataset passado por parâmetro. Primeiro nós criamos o modelo e o adicionamos a uma váriavel depois treinamos esse modelo e o retornamos, diversas variáveis podem ser passadas ao iniciar o treinamento, como imgsz para redimencionar as imagens e agilizar o treinamento, batch para controle da memória ram, device para utilização de GPUs caso estejam disponiveis, etc.

In [8]:
def train_model(dataset_path):
    """Trains the YOLOv8 model."""
    model = YOLO('yolov8m.pt')
    model.train(data=os.path.join(dataset_path, 'data.yaml'), epochs=25)
    return model

## Definição da função de identificação

Aqui criamos a função que realizará a identificação dos objetos, basicamente a função recebe o modelo que deve utilizar e também o caminho para o video (que também decidimos deixar no Google Drive para fácil acesso), a função cv2 captura o video e faz a leitura de frame por frame, passando cada um ao modelo e esperando a resposta sobre se existe ou não um dos objetos que ele foi treinado para identificar na imagem, caso exista, os bounding data são processados e um desenho é feito sobre o frame no local onde foi identificado o objeto, o objeto é salvo na pasta de results do dataset.

In [9]:
def detect_objects(model, video_path):
    cap = cv2.VideoCapture(video_path)
    object_detected = False
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                if class_name in CLASSES_TO_DETECT:
                    object_detected = True
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Pega as coordenadas do bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Desenha bounding box
                    cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Adiciona a label
                    frame_filename = f"frame_{frame_count}.jpg"  # Criar o nome do arquivo do frame
                    frame_path = os.path.join(output_folder, frame_filename)
                    cv2.imwrite(frame_path, frame)  # Salva o frame
                    frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

    return object_detected

## Definição da função de envio de e-mail

Aqui definimos o envio do e-mail quando um objeto cortante é identificado pela função de detecção. Aqui fazemos a validação do email tanto de quem está enviando quanto de quem irá receber utilizando a função validate_email, após validação o e-mail é enviado com assunto e conteudo contento aviso sobre objetos cortantes detectados. Também adicionamos alguns tratamentos de erros para a validação de email e caso outro problema ocorra durante o envio.

In [11]:
def send_email(object_detected):
    if object_detected:
         try:
            # Valida emails e os utiliza caso validados.
            validated_sender = validate_email(SENDER_EMAIL)
            validated_receiver = validate_email(RECEIVER_EMAIL)
            sender_email = validated_sender["email"]
            receiver_email = validated_receiver["email"]

            yag = yagmail.SMTP(user=sender_email, password=SENDER_PASSWORD)
            contents = ["Alerta, objeto cortante e possívelmentet perigoso identificado pelas câmeras de monitoramento."]  # Email content
            yag.send(to=receiver_email, subject="Alerta, objeto perigoso detectado!", contents=contents)
            print("Email sent successfully!")

         except EmailNotValidError as e:
            print(f"Invalid email address: {e}")
         except Exception as e:
             print(f"Error sending email: {e}")
    else:
        print("No sharps objects detected.")

## Rodando o treinamento

Nesse passo realizamos o treinamento do modelo chamando a função que definimos acima, enviando o caminho do dataset e o local onde deve ser salvo.

In [6]:
# model = train_model(dataset_path)
# model.save(os.path.join(dataset_path, 'best.pt')) # Save the trained model

# print("Training finished.")

100%|██████████| 49.7M/49.7M [00:00<00:00, 134MB/s]


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fa

100%|██████████| 755k/755k [00:00<00:00, 25.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 98.0MB/s]


...

      25/25      6.82G      1.103     0.8531      1.464         11        640: 100%|██████████| 369/369 [03:13<00:00,  1.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  1.96it/s]

                   all        468        485      0.592      0.753      0.706      0.376



25 epochs completed in 1.745 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:10<00:00,  1.48it/s]


                   all        468        485      0.601      0.738      0.706      0.376
Speed: 0.3ms preprocess, 10.0ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/train
Training finished.


## Rodando a detecção e o envio de e-mail

Aqui utilizamos o modelo treinado para detecção de objetos cortantes, passando o modelo e também o caminho para o video que será utilizado na validação.

Após a validação, caso algum objeto cortante seja identificado, um email é enviado para a central de segurança.

In [12]:
# 2. Load the trained model
model = YOLO(os.path.join(dataset_path, 'best.pt'))  # Load the trained model. Make sure the 'best.pt' file is in the correct directory.

# 3. Detect objects in the video
object_detected = detect_objects(model, video_path)

# print(object_detected)
# 4. Send email notification
send_email(object_detected)
print("Detection finished.")


0: 384x640 (no detections), 25.3ms
Speed: 2.7ms preprocess, 25.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.6ms
Speed: 3.3ms preprocess, 24.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.8ms
Speed: 3.4ms preprocess, 21.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.8ms
Speed: 3.3ms preprocess, 21.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.6ms
Speed: 3.2ms preprocess, 21.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.3ms
Speed: 2.7ms preprocess, 21.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.4ms
Speed: 3.2ms preprocess, 21.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.4ms
Speed: 3.2ms preprocess, 17.4ms i

<ipython-input-5-9923290be57d>:56: DeprecationWarning: dict-like access to the return value of validate_email is deprecated and may not be supported in the future.
  sender_email = validated_sender["email"]
<ipython-input-5-9923290be57d>:57: DeprecationWarning: dict-like access to the return value of validate_email is deprecated and may not be supported in the future.
  receiver_email = validated_receiver["email"]


Email sent successfully!
Detection finished.
